# Source

> Source code for **y**et**a**nother**s**olve**i**t

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

/opt/conda/lib/python3.12/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


In [ ]:
#| export
from ipylab import JupyterFrontEnd
import openai
import json

class JupyterChat:
    """Integrates a chatbot into JupyterLab, allowing users to interact with an OpenAI model directly within notebooks."""
    def __init__(self, 
                 api_key : str,  # api key for the openai api
                 openai_base_url : str =None, # base url of the openai api
                 model : str =None, # model id for the openai api
                 tag_user : str ='#| chat_user', # tag for user chat markdown cells
                 tag_assistant : str ='#| chat_assistant' # tag for assistant chat markdown cells
                ):
        self.client = openai.Client(base_url=openai_base_url, api_key=api_key)
        self.app = JupyterFrontEnd()
        self.model = model if model is not None else 'meta-llama/llama-3.3-8b-instruct:free'
        self.latest_response = None
        self.tag_user = tag_user
        self.tag_assistant = tag_assistant

    def create_new_markdown_cell(self, 
                                 content: str # Markdown content
                                ):
        """Adds a new markdown cell with the given content below"""
        self.app.commands.execute('notebook:insert-cell-below')
        self.app.commands.execute('notebook:replace-selection', { 'text': content})
        self.app.commands.execute('notebook:change-cell-to-markdown')

    def send_query(self, user_prompt: str):
        """Send user prompt to chatbot and insert response as new cell"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": user_prompt}]
        )
        self.latest_response = response
        response_text = response.choices[0].message.content.strip()
        formatted_response = f'{self.tag_assistant}\n\n{response_text}'.strip()
        self.create_new_markdown_cell(formatted_response)

    def get_current_nb(self):
        """Saves the the notebook returns the JSON content"""
        self.app.commands.execute('docmanager:save')
        fn = self.app.sessions.current_session['name']
        with open(fn, 'r') as f:
            current_notebook = json.load(f)
        return current_notebook
    
    def extract_notebook_dialoge(self):
        """Extracts the tagged dialoge form the current notebook, and turns it into a messages list"""
        current_notebook = self.get_current_nb()
        tmp_messages = []
        for cell in current_notebook['cells']:
            user_tag =  tag_in_cell(cell, self.tag_user)
            ast_tag =  tag_in_cell(cell, self.tag_assistant)
            if any([user_tag, ast_tag]):
                role = 'user' if user_tag else 'assistant'
                tmp_content = ''.join(cell['source'])
                tmp_messages.append({"role": role, "content": tmp_content})
            if all([user_tag, ast_tag]):
                tmp_content = '## ⚠⚠⚠ cell contains user and asssitant tags ⚠⚠⚠\n'
                tmp_content += 'identify the following cell and select only one tag\n'
                tmp_content += '> ' + '> '.join(cell['source'])
                self.create_new_markdown_cell(tmp_content)
        return tmp_messages

    def send_dialoge(self):
        """Sends the dialoge to the openai api and adds the response as a new cell below"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages= self.extract_notebook_dialoge()
        )
        self.latest_response = response
        response_text = response.choices[0].message.content.strip()
        
        # Format the response with Markdown-like style in a code cell
        formatted_response = f'{self.tag_assistant}\n\n{response_text}'.strip()

        # Insert a new cell below with the assistant's response
        self.create_new_markdown_cell(formatted_response)

In [ ]:
#| export
def tag_in_cell(cell, # Dictonary of a Jupyter Notebook cell
                tag # The tag to search
               )->bool: # True if any line contains the given tag
    """Checks a Jupyter Notebook cells source, if any line starts with the given tag"""
    return any([line.startswith(tag) for line in cell['source']])

## Setup JupyterChat
A few things can be demonstraded without having access to an openai API. For instance getting the dictonary or the current notebook using [ipylab](https://github.com/jtpio/ipylab).

In [ ]:
jc = JupyterChat(api_key='not-a-key')

You can get the dictionary of the current notebook.

In [ ]:
# ToDo: Bugfix
#nb = jc.get_current_nb()
#nb.keys()

And search the source code lines of the cells for a specific tag.

In [ ]:
#for cell in nb['cells'][:5]:
#    print(tag_in_cell(cell, '# Source'))

## Connecting to an Openai API
In order to use all JupyterChat you need to provide the base url for the Openai API (not neccessary if its actually openai.com), and a valid API key of the provider.

The following example is using https://openrouter.ai which gives you free access to multiple open source models.

In [ ]:
import os
api_key = os.environ.get('API_KEY')
jc = JupyterChat(openai_base_url="https://openrouter.ai/api/v1", api_key=api_key)

#| chat_assistant
Hello there I am JupyterChat your witty AI assistant. How can I help you today?

#| chat_user
I am wondering when https://solveit.fast.ai/ will make solveit available

In [ ]:
#jc.extract_notebook_dialoge()

In [ ]:
#jc.send_dialoge()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()